# Normalization and batch correction

## Settings

In [ ]:
# Highly Variable Features options 
min_cells = 5 # This one is mandatory
max_cells = None

# Choose normalization method
norm_methods = ['tfidf', 'total']  # can be 'tfidf' and/or 'total'

# Set number of PCs
subset_pcs = True  #True or False
n_pcs = 10  #If None, PCs are chosen automatically. Set n_pcs to overwrite the automatic selection of PCs.

# Set number of neighbors
n_neighbors = 15  #Default=15

# UMAP related settings 
metacol = 'sample'

# batch correction: If True, several batch correction methods will be performed,
# you can choose the best one after
batch_column = "sample"
perform_batch_correction = True
batch_methods = ["bbknn", "harmony"] # "mnn", "scanorama"
threads = 8

--------------

## Loading packages and setup

In [ ]:
import sctoolbox.tools as tools
import sctoolbox.plotting as pl
import sctoolbox.utils as utils
import scanpy as sc
import episcanpy as epi
import numpy as np
utils.settings_from_config("config.yaml", key="03")

sc.set_figure_params(vector_friendly=True, dpi_save=600, scanpy=False)

## Load anndata 

In [ ]:
# probably updated in future
adata = utils.load_h5ad("anndata_2.h5ad")
display(adata)

----------

## Find highly variable features

In [ ]:
# update number of cells per feature
adata = tools.calculate_qc_metrics(adata, var_type='features')

# get highly variable features
tools.get_variable_features(adata, max_cells, min_cells)

In [ ]:
#Number of variable genes selected
adata.var["highly_variable"].sum()

In [ ]:
pl.violin_HVF_distribution(adata)

-----------

## Normalization

In [ ]:
normalizations = tools.normalize_adata(adata, norm_methods)

--------

### Show PC embedding per method

In [ ]:
_ = pl.compare_embeddings(normalizations.values(), adata_names=normalizations.keys(),
                          var_list=[batch_column], embedding="PCA")

In [ ]:
# PCA correlations with obs and var variables
for method, adata_norm in normalizations.items():
    
    _ = pl.plot_pca_correlation(adata_norm, which="obs", 
                                title=f"Normalization method = {method}\nCorrelation of .obs columns with PCA loadings",
                                save=f"PCA_{method}_correlation_obs.pdf")
    
    _ = pl.plot_pca_correlation(adata_norm, which="var", 
                                title=f"Normalization method = {method}\nCorrelation of .var columns with PCA loadings",
                                save=f"PCA_{method}_correlation_var.pdf")

### Remove PC1 for TF-IDF normalization

In [ ]:
normalizations["tfidf-noPC1"] = normalizations["tfidf"].copy()
tools.subset_PCA(normalizations["tfidf-noPC1"], 50, start=1)

--------

### Calc standard neighbors and umap for each adata

In [ ]:
for adata in normalizations.values():
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, method='umap', metric='euclidean')

In [ ]:
tools.wrap_umap(normalizations.values())

### Compare between normalizations

In [ ]:
#Plot the overview of batch correction methods
_ = pl.anndata_overview(normalizations, 
                        plots=["PCA", "PCA-var", "UMAP"],
                        color_by=["n_features"], 
                        output=None)

In [ ]:
# Choose the final normalization method
norm_method = "tfidf-noPC1"

adata = normalizations[norm_method]

-----------

## Select a subset of total PCs

In [ ]:
# Subset the number of pcs if chosen in the parameters
if subset_pcs:
    
    #Automatically identify n_pcs 
    if n_pcs is None:
        n_pcs = tools.define_PC(adata)

# Plot and select number of PCs
_ = pl.plot_pca_variance(adata, save="PCA_variance_selected.pdf", n_pcs=50, n_selected=n_pcs)
tools.subset_PCA(adata, start=0, n_pcs=n_pcs)

## Plot final PCA

In [ ]:
sc.pl.pca_overview(adata, color=['nb_features'], show=False)

-----------

## Batch Correction

In [ ]:
if perform_batch_correction:
    batch_corrections = tools.wrap_corrections(adata, 
                                                  batch_key=batch_column,
                                                  methods=batch_methods)
else:
    batch_corrections = {"uncorrected": adata}

In [ ]:
#Run standard umap for all adatas
tools.wrap_umap(batch_corrections.values(), threads=threads)

In [ ]:
# Should preliminary clustering be performed?
do_clustering = True #True or False

In [ ]:
#Perform additional clustering if it was chosen
color_by = []
if do_clustering:
    for adata in batch_corrections.values():
        sc.tl.leiden(adata, 0.1)
    color_by.append("leiden")

In [ ]:
# Calculate LISI scores for batch
tools.wrap_batch_evaluation(batch_corrections, batch_key=batch_column, threads=threads, inplace=True)

In [ ]:
#Plot the overview of batch correction methods
adata.obs[batch_column] = adata.obs[batch_column].astype("category") #ensure that batch column is a category

_ = pl.anndata_overview(batch_corrections, color_by=color_by + [batch_column], 
                        output=None)

In [ ]:
#Choose an anndata object to proceed
batch_name = "bbknn"

adata_corrected = batch_corrections[batch_name]

---------

## Save anndata

In [ ]:
#Saving the data
adata_output = "anndata_3.h5ad"
utils.save_h5ad(adata, adata_output)